# Laden der Daten
In diesem Notebook werden die Standorte der schweizer Handyantennen aus der Datenquelle data.geo.admin.ch eingelesen. Überflüssige spalten werden entfernt, koordinaten werden ins WSG84 format umgewandelt. Und die Adressdaten werden per Nominatim abgerufen, verarbeitet und im Dataframe erweitert

## Verwendete Links/Quellen

Letzter Zugriff: 01.12.2021

In [1]:
url_5g="http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-5g/data/ch.bakom.mobil-antennenstandorte-5g_de.json"
url_lte='http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-lte/data/ch.bakom.mobil-antennenstandorte-lte_de.json'
url_umts='http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-umts/data/ch.bakom.mobil-antennenstandorte-umts_de.json'
url_gsm='http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-gsm/data/ch.bakom.mobil-antennenstandorte-gsm_de.json'
url_plz = 'https://swisspost.opendatasoft.com/explore/dataset/plz_verzeichnis_v2/table/?disjunctive.postleitzahl&geofilter.distance='


***

# Installation notwendiger Bibliotheken

In [1]:
!pip install --upgrade pip
!pip install folium
!pip install matplotlib
!pip install geopandas
!pip install geopy
!pip install pyproj
!pip install certifi
!pip install ssl


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 2.1/2.1 MB 6.9 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


...


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached ssl-1.16.tar.gz (33 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [25 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 14, in <module>
    File "C:\ProgramData\Anaconda3\lib\site-packages\setuptools\__init__.py", line 189, in <module>
      monkey.patch_all()
    File "C:\ProgramData\Anaconda3\lib\site-packages\setuptools\monkey.py", line 99, in patch_all
      patch_for_msvc_specialized_compiler()
    File "C:\ProgramData\Anaconda3\lib\site-packages\setuptools\monkey.py", line 169, in patch_for_msvc_specialized_compiler
      patch_func(*msvc14('_get_vc_env'))
    File "C:\ProgramData\Anaconda3\lib\site-packages\setuptools\monkey.py", line 149, in patch_params
      mod = import_module(mod_name)
    File "C:\ProgramData\Anaconda3\lib\importlib\__init__.py", line 127, in import_module
      return _bootstrap._gcd_import(name[level:], package, level)
   

***

# Module importieren

In [2]:
import csv

# library to handle data in a vectorized manner 
import numpy as np 

# library for data analsysis
import pandas as pd

import geopandas as gpd

# library to handle JSON files
import json

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

import folium
from folium import plugins

# library to handle requests
import requests

import zipfile

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# necessary for loading url data
import io

from geopy.geocoders import Nominatim

import ssl
import certifi

import geopy.geocoders

import time


from pyproj import Proj, transform

print('Libraries imported.')



from geopy.exc import GeocoderTimedOut

from shapely.geometry import Point, LineString

Libraries imported.


***

# Functions

In [3]:
# https://www.swisstopo.admin.ch/content/swisstopo-internet/de/online/calculation-services/_jcr_content/contentPar/tabs/items/dokumente_und_publik/tabPar/downloadlist/downloadItems/8_1467103085694.download/refsys_d.pdf)
# Zugriff: 08.04.2023
def convert_lv95_wgs84(e,n): #Funktion Umrechnung lv95 zu wgs84
    
    y = (e - 2600000)/1000000
    x = (n - 1200000)/1000000
    
    l = 2.6779094 + 4.728982 * y + 0.791484 * y * x + 0.1306 * y * x**2 - 0.0436 * y**3
    b = 16.9023892 + 3.238272 * x - 0.270978 * y**2 - 0.002528 * x**2 - 0.0447 * y**2 * x - 0.0140 * x**3
    
    laenge_wgs84 = l * 100 / 36
    breite_wgs84 = b * 100 / 36
    
    return breite_wgs84, laenge_wgs84

Die Klasse load_data wurde implementiert, um alle Daten mit einem Befehl einlesen zu können. <br>


In [4]:
class load_data:
    def __init__(self,path):
        self.path=path

    @staticmethod
    def load_from_url(url):
        s=requests.get(url).content
        df=pd.read_csv(io.StringIO(s.decode('utf-8')))
        return df

        
    @staticmethod
    def load_pkl(path_to_pkl):
        with open(path_to_pkl,'rb') as f:
            data=pickle.load(f)
        return data
        
    @staticmethod
    def load_csv(path_to_csv):
        df=pd.read_csv(path_to_csv,sep=',')
        return df

***

# convert coordinates of a dataframe from LV95 to WGS84


In [38]:
# LV95 zu WGS84
def convert_panda_lv95_wgs84(dataFrame ): #Funktion Umrechnung lv95 zu wgs84
    inProj = Proj('epsg:2056')
    outProj = Proj('epsg:4326')
    t = dataFrame['cordinates'].x
    x1,y1 = dataFrame['cordinates'].x, dataFrame['cordinates'].y
    x2,y2 = transform(inProj, outProj, x1,y1)
    dataFrame['antenna_Location_WGS84_E'] = x2
    dataFrame['antenna_Location_WGS84_N'] = y2
    return dataFrame

In [24]:
def get_adress(coordinates, attempt=1, max_attempts=7):
    try:
        locator = Nominatim(user_agent="add_location_adress")
        location = locator.reverse(coordinates)
        address = location.address.split(",")
        countries = address[len(address)-1].split("/")
        address[len(address)-1] = countries[0]
        return address
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_kreis(coordinates, attempt=attempt+1)
        raise

In [61]:
def add_address(data_frame ):

    data_frame['building'] = None
    data_frame['streetNumber'] = None
    data_frame['street'] = None
    data_frame['town'] = None
    data_frame['district'] = None
    data_frame['state'] = None


    street_number = []
    street = []
    town = []
    postal = []
    district = []
    state = []
    country = []

    counter = 0
    for index, row in data_frame.iterrows():
        coordinates = [row['antenna_Location_WGS84_E'],row["antenna_Location_WGS84_N"]]
        forward = 0

        adresse = get_adress(coordinates)

        print(adresse)
        if adresse[forward][0].isdigit() or len(adresse[forward]) >1 and adresse[forward][1].isdigit():
            data_frame.loc[index , "streetNumber"] = adresse[0]
            street_number.append(adresse[0])
            forward += 1
        elif adresse[forward+1][0].isdigit() or len(adresse[forward+1]) >1 and adresse[forward+1][1].isdigit():
            data_frame.loc[index , "streetNumber"] = adresse[forward+1]
            data_frame.loc[index , "building"] = adresse[forward]
            forward += 2

        else:
            street_number.append(None)


        street.append(adresse[forward])
        data_frame.loc[index , "street"] = adresse[forward]


        backward = len(adresse)-1

        country.append(adresse[backward])
        data_frame.loc[index , "country"] = adresse[backward]

        backward -= 1





        if len(adresse[backward]) == 5 and adresse[backward][1].isdigit():
            data_frame.loc[index , "postal"] = adresse[backward]
            data_frame.loc[index , "town"] = plz.get(int(adresse[backward]))
        elif len(adresse[backward+1]) == 5 and adresse[backward+1][1].isdigit():
            data_frame.loc[index , "postal"] = adresse[backward+1]
            data_frame.loc[index , "town"] = plz.get(int(adresse[backward+1]))
            backward = backward + 1
        elif len(adresse[backward-1]) == 5 and adresse[backward-1][1].isdigit():
            data_frame.loc[index , "postal"] = adresse[backward-1]
            data_frame.loc[index , "town"] = plz.get(int(adresse[backward-1]))
            backward = backward - 1


        backward += 1


        postal.append(adresse[backward])







        backward -= 1


        state.append(adresse[backward])
        data_frame.loc[index , "state"] = adresse[backward]
        backward -= 1


        if adresse[backward].__contains__("ezirk") or adresse[backward].__contains__("istrict") or adresse[backward].__contains__("istretto") or adresse[backward].__contains__("erwaltungsregion") or adresse[backward].__contains__("égion administrative") or adresse[backward].__contains__("ahlkreis") or adresse[backward].__contains__("Region"):
            district.append(adresse[backward])
            data_frame.loc[index , "district"] = adresse[backward]
            backward -= 1
        else:
            district.append(None)

        town.append(adresse[backward])

        backward -= 1






        counter += 1
        if counter % 150 == 0:
            time.sleep(30)

        if counter % 9000 == 0:
            time.sleep(1800)


    return data_frame






# 1) Import mapping dictionary for postal codes
<br>Dieser Datensatz liefert dieschweizer Postleitzahlen mit den dazugehöhrigen Ortsnamen dazaus wird ein dictionary erstellt, um später alle Ortsnamen nachzuschlagen <br/>

In [26]:
plz=pd.read_csv('./Daten/plz_verzeichnis_v2.csv',sep=';')
plz = plz[['POSTLEITZAHL', 'ORTBEZ18']]
plz = plz.rename({'POSTLEITZAHL': 'PLZ', 'ORTBEZ18': 'ORT'}, axis=1)  # new method
plz = plz.set_index('PLZ').to_dict()['ORT']


In [27]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-6g_de.json'
antennen_6g = gpd.read_file(data_path)
antennen_6g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2688674.000 1211488.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Sehr Klein,POINT (2666065.000 1214355.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2722651.000 1271244.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2570323.000 1164067.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Klein,POINT (2613427.000 1262637.000)


# Import locations of antena
<br> JSON from antenna to Pandas <br>

In [28]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-5g_de.json'
antennen_5g = gpd.read_file(data_path)
antennen_5g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2688674.000 1211488.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Sehr Klein,POINT (2666065.000 1214355.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2722651.000 1271244.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2570323.000 1164067.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Klein,POINT (2613427.000 1262637.000)


In [29]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-lte_de.json'
antennen_4g = gpd.read_file(data_path)
antennen_4g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Gross,POINT (2558525.000 1167925.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Gross,POINT (2648322.000 1246675.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Gross,POINT (2503428.000 1117642.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2601088.000 1200806.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Gross,POINT (2567350.000 1228990.000)


In [30]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-umts_de.json'
antennen_3g = gpd.read_file(data_path)
antennen_3g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2601088.000 1200806.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Gross,POINT (2639748.000 1232218.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2613427.000 1262637.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2503428.000 1117642.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Klein,POINT (2570323.000 1164067.000)


In [31]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-gsm_de.json'
antennen_2g = gpd.read_file(data_path)
antennen_2g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2701702.000 1265811.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Mittel,POINT (2618398.000 1227312.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2609644.000 1213344.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2560331.000 1182192.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Sehr Klein,POINT (2557146.000 1210825.000)


# Prepare Datasets
<br> Set column names and drop usless Columns <br>

In [39]:

columns=['language', 'description', 'id', 'power', 'cordinates']
antennen_6g.columns=columns
antennen_5g.columns=columns
antennen_4g.columns=columns
antennen_3g.columns=columns
antennen_2g.columns=columns

ValueError: Length mismatch: Expected axis has 9 elements, new values have 5 elements

In [33]:
antennen_6g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_5g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_4g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_3g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_2g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_4g.head(4)

,id,power,cordinates
0,1,Gross,POINT (2558525.000 1167925.000)
1,2,Gross,POINT (2648322.000 1246675.000)
2,3,Gross,POINT (2503428.000 1117642.000)
3,4,Mittel,POINT (2601088.000 1200806.000)


# get address data from openstreet map

In [36]:
import ssl
import certifi
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())

C:\ProgramData\Anaconda3\lib\site-packages\certifi\cacert.pem


In [40]:
antennen_6g = convert_panda_lv95_wgs84(antennen_6g)
antennen_5g = convert_panda_lv95_wgs84(antennen_5g)
antennen_4g = convert_panda_lv95_wgs84(antennen_4g)
antennen_3g = convert_panda_lv95_wgs84(antennen_3g)
antennen_2g = convert_panda_lv95_wgs84(antennen_2g)




C:\Users\nikki\AppData\Local\Temp\ipykernel_23640\4242243979.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj, outProj, x1,y1)
C:\Users\nikki\AppData\Local\Temp\ipykernel_23640\4242243979.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj, outProj, x1,y1)
C:\Users\nikki\AppData\Local\Temp\ipykernel_23640\4242243979.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj, outProj, x1,y1)
C:\Users\nikki\AppData\Local\Temp\ipykernel_23640\4242243979.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj, outProj, x1,y1)
C:\U

In [62]:
antennen_6g = add_address(antennen_6g)
# antennen_5g = add_address(antennen_5g)
# antennen_4g = add_address(antennen_4g)
# antennen_3g = add_address(antennen_3g)
# antennen_2g = add_address(antennen_2g)

['Frauholzstrasse', ' Oberes Halti', ' Steinen', ' Schwyz', ' 6422', ' Schweiz']
6422
Steinen Zustellung
['A14', ' Milchhof', ' Ebikon', ' Luzern', ' 6030', ' Schweiz']
6030
Ebikon Mall Switz.
['Muggenwinggel', ' Weinfelderstrasse', ' Feldhof', ' Märstetten', ' Bezirk Weinfelden', ' Thurgau', ' 8560', ' Schweiz']
8560
Märstetten
['71', ' Rue de la Léchère', ' la Léchère', ' Bulle', ' District de la Gruyère', ' Fribourg/Freiburg', ' 1630', ' Schweiz']
1630
Bulle Jardins P.L.
['27', ' Weidenstrasse', ' Münchenstein', ' Bezirk Arlesheim', ' Basel-Landschaft', ' 4142', ' Schweiz']
4142
Münchenstein Stöck
['Outletpark Murgenthal', ' 79', ' Hauptstrasse', ' Moosmattdorf', ' Riken', ' Murgenthal', ' Bezirk Zofingen', ' Aargau', ' 4853', ' Schweiz']
4853
Riken AG
['Krummackerweg', ' Leibstadt', ' Bezirk Zurzach', ' Aargau', ' 5325', ' Schweiz']
5325
Leibstadt
['3', ' Chemin des Coquelicots', ' Châtelaine', ' Vernier', ' Genève', ' 6643', ' Schweiz']
6643
None
['12', ' Bruggmatte', ' Reiden', '

In [64]:



# Rename special chars
def rename_Special(data_frame):
    data_frame = data_frame.replace({'dictionary':{'ä':'ae','Ä':'Ae','Ü':'Ue','ü':'ue','Ö':'Oe','ö':'oe'}},regex=True)
    return data_frame

In [65]:
antennen_6g = rename_Special(antennen_6g)
antennen_5g = rename_Special(antennen_5g)
antennen_4g = rename_Special(antennen_4g)
antennen_3g = rename_Special(antennen_3g)
antennen_2g = rename_Special(antennen_2g)

In [82]:

antennen_6g = antennen_6g.assign(type='6G')
antennen_5g = antennen_6g.assign(type='5G')
antennen_4g = antennen_6g.assign(type='4G')
antennen_3g = antennen_6g.assign(type='3G')
antennen_2g = antennen_6g.assign(type='2G')
antennen_6g.head()


,id,power,cordinates,building,streetNumber,street,town,district,state,antenna_Location_WGS84_E,antenna_Location_WGS84_N,country,postal,type
0,1,Mittel,POINT (2688674.000 1211488.000),None,None,Frauholzstrasse,Steinen Zustellung,None,6422,47.048492,8.605623,Schweiz,6422,6G
1,2,Sehr Klein,POINT (2666065.000 1214355.000),None,A14,Milchhof,Ebikon Mall Switz.,None,6030,47.076917,8.308525,Schweiz,6030,6G
2,3,Mittel,POINT (2722651.000 1271244.000),None,None,Muggenwinggel,Märstetten,None,8560,47.580429,9.069008,Schweiz,8560,6G
3,4,Mittel,POINT (2570323.000 1164067.000),None,71,Rue de la Léchère,Bulle Jardins P.L.,None,1630,46.627189,7.051134,Schweiz,1630,6G
4,5,Klein,POINT (2613427.000 1262637.000),None,27,Weidenstrasse,Münchenstein Stöck,None,4142,47.514341,7.616871,Schweiz,4142,6G


In [32]:
# antennen_5g.to_csv('./Daten/5gAntennen.csv',sep=';', index=False)
antennen_4g.to_csv('./Daten/4gAntennen.csv',sep=';', index=False)
antennen_3g.to_csv('./Daten/3gAntennen.csv',sep=';', index=False)
antennen_2g.to_csv('./Daten/2gAntennen.csv',sep=';', index=False)

In [74]:
print(certifi.where())
#Breiten- und Längengrad von Zürich
address = 'Schweiz'

geolocator = Nominatim(user_agent="antenna")
location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude

C:\ProgramData\Anaconda3\lib\site-packages\certifi\cacert.pem


Erster Plot zum anschauen des Netzes:

In [75]:
# workaround for SSL Error:
import ssl
import certifi
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())

C:\ProgramData\Anaconda3\lib\site-packages\certifi\cacert.pem


In [84]:
m=folium.Map(location=[latitude,longitude],zoom_start=9)

In [86]:


color = 'white'

if antennen_6g['type'][0] == '2G':
    color = 'blue'
if antennen_6g['type'][0] == '3G':
print(antennen_6g['type'][0])

#You Markler the point in Map
for indice, row in antennen_6g.iterrows():
    folium.Marker(
        location=[row['antenna_Location_WGS84_E'], row['antenna_Location_WGS84_N']],
        # popup=row,
        popup=[row['building'] , row["streetNumber"] ,row['postal'],  row['town'] , row['district'] , row['state'] ],
        icon=folium.map.Icon(color='green')
    ).add_to(m)
m

6G


In [69]:
folium.Map(location= [ antennen_6g['antenna_Location_WGS84_E'] , antennen_6g['antenna_Location_WGS84_N']])
# folium.Choropleth(gdf_routennetz[gdf_routennetz.velostreifen!='0'],line_color='blue',line_weight=5,line_opacity=0.3,columns=['name'],legend_name='hehe').add_to(m)
# folium.Choropleth(gdf_routennetz[gdf_routennetz.veloweg!=0].geometry,line_color='red',line_weight=5,line_opacity=0.3).add_to(m)

ValueError: Location should consist of two numerical values, but 0     47.048492
1     47.076917
2     47.580429
3     46.627189
4     47.514341
5     47.271302
6     47.591917
7     46.220723
8     47.239688
9     46.527151
10    47.096465
11    46.187212
12    47.482354
13    47.389119
14    47.394254
15    47.404670
16    46.067722
17    46.716377
18    46.936643
19    46.866157
20    46.617885
21    47.380211
22    46.974817
23    46.500624
Name: antenna_Location_WGS84_E, dtype: float64 of type <class 'pandas.core.series.Series'> is not convertible to float.

In [2]:

display(m)

NameError: name 'm' is not defined

***

# 4) Abbiegeverbote Stadt Zürich
<br>Die Abbiegeverbote geben an, wo man als Velofahrerin nicht abbiegen darf.<br/>
<br>Zugriff: 01:11:2021/20:05 https://data.stadt-zuerich.ch/dataset/geo_fuss__und_velowegnetz<br/>

In [75]:
gdf_abbiegeverbot = gpd.read_file('./Daten/Fuss_Velo_Wegnetz/data/taz_mm.tbl_routennetz_abbiegeverbote.json')
gdf_abbiegeverbot.head()

,id1,se_anno_cad_data,objectid,geometry
0,108,NaN,1,"LINESTRING (8.54526 47.36752, 8.54522 47.36750..."
1,109,NaN,2,"LINESTRING (8.54259 47.37874, 8.54255 47.37867)"
2,110,NaN,3,"LINESTRING (8.55336 47.40247, 8.55345 47.40251..."
3,111,NaN,4,"LINESTRING (8.50459 47.40565, 8.50460 47.40563..."
4,112,NaN,5,"LINESTRING (8.53681 47.38516, 8.53679 47.38521..."


In [76]:
gdf_abbiegeverbot.drop(['se_anno_cad_data','id1','objectid'], axis=1, inplace=True)

In [77]:
gdf_abbiegeverbot.head()

,geometry
0,"LINESTRING (8.54526 47.36752, 8.54522 47.36750..."
1,"LINESTRING (8.54259 47.37874, 8.54255 47.37867)"
2,"LINESTRING (8.55336 47.40247, 8.55345 47.40251..."
3,"LINESTRING (8.50459 47.40565, 8.50460 47.40563..."
4,"LINESTRING (8.53681 47.38516, 8.53679 47.38521..."


In [78]:
gdf_abbiegeverbot.to_csv('./Daten/abbiegeverbote_ZH.csv', index=False)